# 파일 불러오기

In [1]:
from google.colab import files
myfile = files.upload()

Saving 국회(전처리).csv to 국회(전처리).csv


In [2]:
import io
import pandas as pd

In [3]:
data = pd.read_csv(io.BytesIO(myfile['국회(전처리).csv']))
data.head()

,title,count,categ,start,text
0,국가보안법 개정에 관한 청원,1761,수사/법무/사법제도,2021-06-16,"['최근', '국가', '보안법', '폐지', '올라오', '자유', '대한민국',..."
1,인공지능 윤리 및 고인의 AI구현 법안 제정에 관한 청원,223,과학기술/정보통신,2021-06-15,"['동의', 'AI', '구현', '제재', '가하', '규정', '만들', '외의..."
2,포괄적 차별금지법 법안 제정에 관한 동의 및 일부 내용 수정 요청에 관한 청원,14025,인권/성평등/노동,2021-06-15,"['UN', '사회권', '위원회', '국가', '인권', '위원회', '포괄적',..."
3,국민연금 수급 개시 연령과 연계한 정년연장에 관한 청원,19807,인권/성평등/노동,2021-06-14,"['노동자', '정년', '기간', '국민', '연금', '수급', '개시', '연..."
4,하수도법 일부개정법률안에 관한 청원,736,재난/안전/환경,2021-06-11,"['하수도', '특정', '공산품', '주방', '오물', '분쇄', '기은', '..."


In [4]:
assembly_df = data.loc[:, ['text']]
print(assembly_df)

                                                  text
0    ['최근', '국가', '보안법', '폐지', '올라오', '자유', '대한민국',...
1    ['동의', 'AI', '구현', '제재', '가하', '규정', '만들', '외의...
2    ['UN', '사회권', '위원회', '국가', '인권', '위원회', '포괄적',...
3    ['노동자', '정년', '기간', '국민', '연금', '수급', '개시', '연...
4    ['하수도', '특정', '공산품', '주방', '오물', '분쇄', '기은', '...
..                                                 ...
215  ['최근', '한국', '심리', '학회', '상담', '심리', '학회', '임상...
216  ['출판계', '플랫폼', '막아주', '한국', '출판계', '미래', '소설',...
217  ['일요일', '공휴일', '대체', '휴일', '일괄적', '다음', '월요일',...
218  ['순기능', '제대로', '역효과', '일으키', '셧다', '운제', '청소년'...
219  ['세계', '민주주의', '여러', '나라', '가운데', '배심', '원제', ...

[220 rows x 1 columns]


# 역토큰화

In [5]:
assembly_df.text = assembly_df.text.str.lstrip("['").str.rstrip("']").str.split("', '").str.join(" ")
print(assembly_df.text)

0      최근 국가 보안법 폐지 올라오 자유 대한민국 월남 전철 무척 우려 스럽 국가 보안법...
1      동의 AI 구현 제재 가하 규정 만들 외의 AI 구현 가능 인공지능 기술 발전 정보...
2      UN 사회권 위원회 국가 인권 위원회 포괄적 차별 금지법 제정 한국 정부 권고 민주...
3      노동자 정년 기간 국민 연금 수급 개시 연령 연계 고용 연령 차별 금지 고령자 고용...
4      하수도 특정 공산품 주방 오물 분쇄 기은 한미 FTA 무역 협정 공산품 특정 공산품...
                             ...                        
215    최근 한국 심리 학회 상담 심리 학회 임상 심리 학회 산하 분과 학회 한국 상담 학...
216    출판계 플랫폼 막아주 한국 출판계 미래 소설 작가 플랫폼 떠넘기 과도 수수료 방지 ...
217    일요일 공휴일 대체 휴일 일괄적 다음 월요일 정하 의견 개천절 한글날 대체 휴일 모...
218    순기능 제대로 역효과 일으키 셧다 운제 청소년 보호법 제조 폐지 바람 셧다운제 시행...
219    세계 민주주의 여러 나라 가운데 배심 원제 국민 참여 재판 가장 국가 하나 한국 이...
Name: text, Length: 220, dtype: object


# TF-IDF 행렬 만들기

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(assembly_df.text.values.astype('U'))
X.shape

(220, 7712)

# 토픽 모델링

In [7]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components = 28)
svd_model.fit(X)
len(svd_model.components_)

28

In [8]:
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n = 10):
  for idx, topic in enumerate(components):
    print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(10)) for i in topic.argsort()[:-n-1:-1]])
get_topics(svd_model.components_, terms)

Topic 1: [('투표', 0.3562806344), ('선거', 0.2378846801), ('사전', 0.2044218275), ('국민', 0.1776727419), ('개표', 0.1372338572), ('여성', 0.1287571502), ('성범죄', 0.1186662845), ('처벌', 0.1181757815), ('부정', 0.1181687155), ('피해자', 0.1093312136)]
Topic 2: [('투표', 0.4493257224), ('선거', 0.2698516023), ('사전', 0.2591160439), ('개표', 0.1551445925), ('부정', 0.1389537595), ('투표함', 0.1357897736), ('보관', 0.1260539261), ('선거법', 0.1134574853), ('공직', 0.108835361), ('전자', 0.1049206019)]
Topic 3: [('코로나', 0.2173671081), ('국민', 0.1568521036), ('통제', 0.1328270273), ('가족', 0.1081283083), ('대통령', 0.1002584272), ('헌법', 0.1000873215), ('탄핵', 0.0996714182), ('검사', 0.0993462173), ('마스크', 0.0968027376), ('검찰', 0.0869741066)]
Topic 4: [('여성', 0.5264013908), ('가족', 0.4105207797), ('남성', 0.180816314), ('평등', 0.1580140051), ('차별', 0.113749602), ('매매', 0.1137277096), ('여가', 0.0965980852), ('부가', 0.0789261532), ('폭력', 0.078692845), ('복지부', 0.0748431714)]
Topic 5: [('게임', 0.5260110016), ('개표', 0.272318778), ('전자', 0.2477135758), (